## 데이터 불러오기

In [1]:
import pandas as pd
import requests

In [2]:
res = requests.get('https://github.com/e9t/nsmc/raw/master/ratings_train.txt')
with open('ratings_train.txt','wb') as f:
    f.write(res.content)

nsmc = pd.read_csv('ratings_train.txt',sep='\t')

In [3]:
nsmc

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


## 전처리

In [4]:
import re

In [34]:
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+',text)

In [35]:
nsmc.loc[0,'document']

'아 더빙.. 진짜 짜증나네요 목소리'

In [36]:
find_hangul(nsmc.loc[0,'document'])

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [37]:
nsmc['document'].notnull()

0         True
1         True
2         True
3         True
4         True
          ... 
149995    True
149996    True
149997    True
149998    True
149999    True
Name: document, Length: 150000, dtype: bool

In [38]:
nsmc[nsmc['document'].notnull()]

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [42]:
data =nsmc[nsmc['document'].notnull()]['document'].map(find_hangul)

In [43]:
data[0]

['아', '더빙', '진짜', '짜증나네요', '목소리']

In [44]:
data[1]

['흠', '포스터보고', '초딩영화줄', '오버연기조차', '가볍지', '않구나']

In [45]:
def only_hangul(text):
    return ' '.join(find_hangul(text))

In [47]:
only_hangul(nsmc.loc[0,'document'])

'아 더빙 진짜 짜증나네요 목소리'

In [49]:
data2 = nsmc[nsmc['document'].notnull()]['document'].map(only_hangul)

In [50]:
with open('nsmc.txt','w',encoding='utf-8') as f:
    f.write('\n'.join(data2))

## FastText 모형 학습

In [54]:
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText

In [55]:
# sb = 0(기본값) : cbow학습, s=1 : skip-gram 학습
# alpha = 0.025 : 학습률
# min_alpha : 학습률을 낮춰주는 역할
# window = 5 : 최소 몇번 나와야 학습을 함
# min_count = 5 : 최소 몇번 나와야 학습을 함
# vector_size = 100 : 기본값
model = FastText(vector_size=16)

In [56]:
model.build_vocab(corpus_iterable=data)

In [57]:
model.train(corpus_iterable=data,
            epochs=5,
            total_examples=model.corpus_count,
            total_words=model.corpus_total_words
)

(3999416, 5829395)

## 저장

In [58]:
model.save('nsmc.fasttext')

## 임베딩

In [59]:
model = FastText.load('nsmc.fasttext')

In [60]:
model.wv.key_to_index

{'영화': 0,
 '너무': 1,
 '정말': 2,
 '진짜': 3,
 '이': 4,
 '그냥': 5,
 '왜': 6,
 '이런': 7,
 '더': 8,
 '점': 9,
 '수': 10,
 '영화를': 11,
 '다': 12,
 '잘': 13,
 '좀': 14,
 '보고': 15,
 'ㅋㅋ': 16,
 '그': 17,
 '영화가': 18,
 '영화는': 19,
 '본': 20,
 '봤는데': 21,
 '최고의': 22,
 '아': 23,
 '이건': 24,
 '내가': 25,
 '드라마': 26,
 '없는': 27,
 '없다': 28,
 '평점': 29,
 '완전': 30,
 '이렇게': 31,
 '참': 32,
 '이거': 33,
 '그리고': 34,
 '이게': 35,
 '좋은': 36,
 '있는': 37,
 '연기': 38,
 '내': 39,
 '평점이': 40,
 '보는': 41,
 '최고': 42,
 '다시': 43,
 '역시': 44,
 '스토리': 45,
 '쓰레기': 46,
 'ㅋ': 47,
 '난': 48,
 '많이': 49,
 '것': 50,
 '한': 51,
 'ㅋㅋㅋ': 52,
 '재밌게': 53,
 '없고': 54,
 '또': 55,
 '하는': 56,
 '아깝다': 57,
 '꼭': 58,
 '보면': 59,
 '마지막': 60,
 '가장': 61,
 '뭐': 62,
 '영화다': 63,
 '무슨': 64,
 '하지만': 65,
 '같은': 66,
 'ㅎㅎ': 67,
 '와': 68,
 '별로': 69,
 '작품': 70,
 '솔직히': 71,
 '끝까지': 72,
 '볼': 73,
 '넘': 74,
 '안': 75,
 '대한': 76,
 '만든': 77,
 '봐도': 78,
 '그래도': 79,
 '시간': 80,
 '같다': 81,
 '전혀': 82,
 '좋다': 83,
 '말이': 84,
 '지금': 85,
 '별': 86,
 '아주': 87,
 '근데': 88,
 '중': 89,
 '뭔가': 90,
 '영화의': 91,
 '하

In [61]:
'히어로' in model.wv.key_to_index

True

In [62]:
model.wv['히어로']

array([-0.4142657 ,  0.6238779 ,  0.5336922 ,  0.41763264,  0.828825  ,
        0.13039324, -0.46707115, -0.5840886 ,  0.1707697 ,  0.10000038,
        0.258987  , -0.89228857,  0.26910207, -0.3837389 , -0.10001663,
        0.36368346], dtype=float32)

In [63]:
'슈퍼히어로' in model.wv.key_to_index

False

In [64]:
model.wv['슈퍼히어로']

array([-0.1793366 ,  0.2602628 ,  0.22193988,  0.19039355,  0.32874137,
        0.08479347, -0.16141792, -0.25373527,  0.11844546,  0.03226747,
        0.14068785, -0.3316593 ,  0.06700411, -0.11679079, -0.00265816,
        0.1890587 ], dtype=float32)

## 유사도

In [65]:
model.wv.similarity('슈퍼히어로','히어로') # 코사인 유사도로 비교

0.98815584

In [66]:
model.wv.similarity('히어로','평론가')

0.71533275

In [67]:
model.wv.most_similar('평론가')

[('기자', 0.9896329641342163),
 ('점대나', 0.9890762567520142),
 ('높은거야', 0.9868955016136169),
 ('평론', 0.9868379235267639),
 ('점이야', 0.9857552647590637),
 ('점이라', 0.9857050776481628),
 ('전문가', 0.9849140048027039),
 ('점이나', 0.9847507476806641),
 ('점대가', 0.984673798084259),
 ('점대지', 0.9844580292701721)]

## FastText를 이용한 감성분석

In [68]:
from gensim.models.fasttext import FastText

ft = FastText.load('nsmc.fasttext')

In [69]:
import pandas as pd

nsmc = pd.read_csv('ratings_train.txt',sep='\t')

In [70]:
df = nsmc[nsmc['document'].notnull()]

In [71]:
df

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [72]:
from sklearn.model_selection import train_test_split

doc_train, doc_test, y_train, y_test = train_test_split(df['document'],df['label'],test_size=0.2,random_state=42)

In [73]:
import re
def find_hangul(text):
    return re.findall(r'[ㄱ-ㅎ가-힣]+',text)

In [74]:
import numpy as np
x_train = np.zeros((1000,16))

In [75]:
doc_train.shape

(119996,)

In [77]:
doc_train

31989            아 꿀잼ㅋ 친구랑 봤는데 너무 웃겼음 그리구 김우빈 잘생겼다..
63462                    개건의 졸작 스릴러? 스릴러라고 하게에도 민망하군
17518               장하나 한윤찬 제발 이어주세요 말도안되게 왜 설도현과ㅜㅜㅜ
123410                                   애로영화계의 개OOO
104181                               내용이나 그래픽자체가 허접함
                             ...                    
119882      꿈을 꾸는사람 꿈을 이룬사람 돌멩이도 꿈은 있잖아! 꿈과희망을 주는 영화
103696                                레니 할린.. 이게 뭐니?
131936                                    시라노; 연애조작단
146872                                    집중이 쉽지 않다.
121961    엔딩 장면이 좋고 소소한 일상과 고민과 연애가 공감을 주어 재밌게 보았어여~
Name: document, Length: 119996, dtype: object

In [97]:
for i,doc in enumerate(doc_train[:1000]):
    vs = [ft.wv[word] for word in find_hangul(doc) if word in ft.wv]
    if vs:
        # print(doc)
        # print(len(vs))
        # print(vs[0])
        x_train[i,] = np.mean(vs,axis=0)
    else:
        print(doc)

once upon a dream
good
good
it's jjangge movie
Good message
